In [10]:
!pip install pycm livelossplot
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'Normalize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [13]:
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import Dataset 
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, ToPILImage, RandomResizedCrop, RandomAffine
import random
import itertools


def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

No GPU available!


##Define Train, Validate, Test Functions



In [0]:
def train(model, optimizer, criterion, data_loader):
    model.train()
    train_loss, train_accuracy = 0., 0.
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        
        train_loss += loss*X.size(0)
        train_accuracy += r2_score(y.cpu().numpy(), y_pred.detach().cpu().numpy())*X.size(0)
  
        optimizer.step()
        
    return train_loss/len(data_loader.dataset), train_accuracy/len(data_loader.dataset)
  
def validate(model, criterion, data_loader):
    model.eval()
    validation_loss, validation_accuracy = 0., 0.
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            
            output = model(X)

            loss = criterion(output, y)
            validation_loss += loss*X.size(0)
            validation_accuracy += r2_score(y.cpu().numpy(), y_pred.detach().cpu().numpy())*X.size(0)
  
    return validation_loss/len(data_loader.dataset), validation_accuracy/len(data_loader.dataset)
  
def evaluate(model, data_loader):
    model.eval()
    ys, y_preds = [], []
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            
            output = model(X)
            
#             y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            ys.append(y.cpu().numpy())
            y_preds.append(y_pred.cpu().numpy())
            
    return np.concatenate(y_preds, 0),  np.concatenate(ys, 0)

## Model Development

In [0]:
class model1(nn.Module):
  def __init__(self):
    super(model1, self).__init__()
    self.input = nn.Linear(1,5)
    self.linear1 = nn.Linear(5,5)
    self.output = nn.Linear(5,1)
    self.act = nn.tanh()
    
    
  def forward(self, x):
    
    x = self.act(self.input(x))
    x = self.act(self.linear1(x))
    
    
    x = self.act(self.output(x))
    
    return x

## Import Data

In [0]:
# Input data should be [mx1]


train_data = 
validate_data = 
test_data = 

## Train and Validate Model

In [0]:
def run(in_model, seed, lr, weight_decay):
    model = in_model().to(device)

    set_seed(seed)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
    criterion = nn.MSELoss()

    liveloss = PlotLosses()
    for epoch in range(n_epochs):
        logs = {}
        train_loss, train_accuracy = train(model, optimizer, criterion, train_loader)

        logs['' + 'log loss'] = train_loss.item()
        logs['' + 'accuracy'] = train_accuracy.item()

        validation_loss, validation_accuracy = validate(model, criterion, test_loader)
        logs['val_' + 'log loss'] = validation_loss.item()
        logs['val_' + 'accuracy'] = validation_accuracy.item()

        liveloss.update(logs)
        liveloss.draw()

    test_loss, test_acc = validate(model, criterion, test_loader)
    print("Avg. Test Loss: %1.3f" % test_loss.item(), " Avg. Test Accuracy: %1.3f" % test_acc.item())
    print("")

# Split data into train and test sets, labels and features
# Change the data!!!!
X_train, y_train = X_train_orig.astype(float), y_train_orig
X_test, y_test = X_test_orig.astype(float), y_test_orig

X_train, y_train = torch.from_numpy(X_train).float(), torch.from_numpy(y_train)
X_test, y_test = torch.from_numpy(X_test).float(), torch.from_numpy(y_test)

# mean, std = torch.mean(X_train), torch.std(X_train)

# # Assemble tensor datasets
# train_ds = CustomImageTensorDataset(X_train, y_train.long(), transform=True, mean=mean, std=std)
# test_ds = CustomImageTensorDataset(X_test, y_test.long(), transform=False, mean=mean, std=std)

# # Assemble dataloaders
# train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
# test_loader = DataLoader(test_ds, batch_size=test_batch_size, shuffle=False, num_workers=0)

run(model, seed=42, lr, weight_decay)